# 03 FRF synthetization

Modal superposition allows easy and fast generation of FRFs from the finite element model. By solving the eigenvalue problem, eigenfrequencies and eigenvectors of the system are obtained. 

In [1]:
import pyFBS

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

## Data import

Importing of example data: 
 - 3D model in stl format
 - locations and directions of responses and impacts in xlsx format
 - Ansys .full file
 - Ansys .res file
 - experimental data from .pickle file

In [2]:
pyFBS.download_lab_testbench()

100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 1752.95it/s]

In [3]:
stl = r"./lab_testbench/STL/B.stl"
xlsx = r"./lab_testbench/Measurements/AM_measurements.xlsx"

full_file = r"./lab_testbench/FEM/B.full"
rst_file = r"./lab_testbench/FEM/B.rst"

exp_file = r"./lab_testbench/Measurements/Y_B.p"

### Experiemntal data import

In [4]:
freq, Y_B_exp = np.load(exp_file,allow_pickle = True)

## Initialisation

Initialisation of MK model. Location of ``.ress`` and ``.full`` file must be defined.

A number of modes that will be taken into account when solving the eigenvalue problem are set in the ``no_modes`` parameter, for faster processing of re-imports, a pickle file can be read (``allow_pickle`` parameter), the repetition of eigenvalue problem solving is set in the ``recalculate`` parameter.

In [5]:
MK = pyFBS.MK_model(rst_file, full_file, no_modes = 100, allow_pickle = False, recalculate = True)

``MK`` parameter is ``pyFBS.MCK.MK_model`` class with following attributes:

In [6]:
MK.__dict__.keys()

dict_keys(['nodes', 'mesh', 'pts', 'no_modes', '_all', 'dof_ref', 'K', 'M', '_K', 'rotation_included', 'eig_freq', 'eig_val', 'eig_vec'])

In [7]:
MK.nodes             # all nodes of finite element model
MK.mesh              # class: pyvista.core.pointset.UnstructuredGrid, where moes of finite element model is defined
MK.pts               #  all nodes to be rendered converted to mm
MK.no_modes          # number of modes taken in to account when solving eigenvalue probelm
MK.dof_ref           # DoFs at specific node
MK.K                 # stiffness matrix in sparse form
MK.M                 # mass matrix in sparse form
MK._K                # temporary stiffness matrix in sparse form for pickle import check
MK.rotation_included # finite element model includes rotation DoFs
MK.eig_freq          # eigenfrequencies of finite element model in [rad/s]
MK.eig_val           # eigenvalues of finite element model in [rad^2/s^2]
MK.eig_vec;          # eigenvectors of finite element model

## 3D view

In [8]:
view3D = pyFBS.view3D(show_origin= True)

In [9]:
view3D.add_stl(stl,name = "engine_mount",color = "#8FB1CC",opacity = .1);

In [10]:
view3D.plot.add_mesh(MK.mesh, scalars = np.ones(MK.mesh.points.shape[0]) ,name = "mesh",cmap = "coolwarm", show_edges = True);

### Animate mode shape

In [11]:
select_mode = 7
_modeshape = MK.get_modeshape(select_mode)

mode_dict = pyFBS.dict_animation(_modeshape,"modeshape",pts = MK.pts, mesh = MK.mesh)
view3D.add_modeshape(mode_dict,run_animation = True)

In [12]:
view3D.clear_modeshape()

### Show accelereometers

In [13]:
df_acc = pd.read_excel(xlsx, sheet_name='Sensors_B')
view3D.show_acc(df_acc,overwrite = True)
#view3D.label_imp(df_imp)
#df_imp

In [14]:
df_chn = pd.read_excel(xlsx, sheet_name='Channels_B')
view3D.show_chn(df_chn)
#df_chn

### Show impacts

In [15]:
df_imp = pd.read_excel(xlsx, sheet_name='Impacts_B')
view3D.show_imp(df_imp,overwrite = True)
#view3D.label_imp(df_imp)
#df_imp

## Update experimental locations to closest numerical location

The locations and directions of responses and excitations often do not match exactly with the numerical model, so we need to find the nodes closest to these points. Only the locations are updated, the directions remain the same.

In [16]:
df_chn_up = MK.update_locations_df(df_chn)
df_imp_up = MK.update_locations_df(df_imp)

In [17]:
view3D.show_chn(df_chn_up, color = "k",overwrite = False)
view3D.show_imp(df_imp_up, color = "k",overwrite = False)

# FRF synthetization

For the synthetization of FRFs, it is necessary to define the locations and directions that we want to be included in the final response model. This is defined in the form of pandas.DataFrame, which must have the following labels: 
 - `` Position_1``, `` Position_2``, `` Position_3`` - for locations
 - `` Direction_1``, `` Direction_2``, `` Direction_3`` - for directions
 
The frequency range and its resolution are set via the parameters ``f_start``, ``f_end`` and ``f_resolution``, the number of modes used for reconstruction can be defined in the parameter ``limit_modes`` (if ``limit_modes = None`` then the same number of models will be used for reconstruction as defined during initialization in the ``no_modes`` parameter) and the constant damping can be set in the ``modal_damping`` parameter. The result of the  FRF synthetization can be in the form of ``accelerance``, ``mobility`` or ``receptance``, which is defined in the ``frf_type`` parameter.
 
If defined locations in given pandas.DataFrames do not coincide with locations in the numerical model, will FRFs be generated at the nearest nodes of numerical model. Nodes, where FRF were actually generated, are defined in parameters ``df_chn_up`` and ``df_imp_up``.

In [18]:
MK.FRF_synth(df_chn,df_imp,f_start = 0,f_end =2000 ,f_resolution = 1, limit_modes = 50, modal_damping = 0.003,frf_type = "accelerance")

## Add noise to FRFs

Random noise can also be added to the synthesized FRFs:

In [19]:
MK.add_noise(n1 = 2e-2, n2 = 2e-1, n3 = 2e-1 ,n4 = 5e-2)

# FRF visualisation

In [20]:
plt.figure(figsize = (8,6))

s1 = 3
s2 = 0

param = ["Name", "Quantity","Position_1", "Position_2", "Position_3", "Direction_1", "Direction_2", "Direction_3"]

df_disp = df_chn.iloc[[s1]][param].copy()
display(df_disp.append(df_imp.iloc[[s2]][param]))

plt.subplot(211)
plt.semilogy(MK.freq,np.abs(MK.FRF_noise[:,s1,s2]), alpha=0.4, label = "Num. FRF + noise")
plt.semilogy(MK.freq,np.abs(MK.FRF[:,s1,s2]), label = "Num. FRF")
plt.semilogy(freq,np.abs(Y_B_exp[s1,s2]), label = "Exp. FRF")
plt.legend()

plt.subplot(413)
plt.plot(MK.freq,np.angle(MK.FRF_noise[:,s1,s2]), alpha=0.4)
plt.plot(MK.freq,np.angle(MK.FRF[:,s1,s2]))
plt.plot(freq,np.angle(Y_B_exp[s1,s2]));

,Name,Quantity,Position_1,Position_2,Position_3,Direction_1,Direction_2,Direction_3
3,Sensor 4,Acceleration,33.938789,369.124840,-7.518755,0.0,0.0,1.0
0,Impact 7,Force,46.035934,323.204458,27.000001,0.0,0.0,-1.0
